In [1]:
# Libraries from 3rd parties
import pandas as pd
import numpy as np
import statistics
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.io as pio
pio.orca.config.executable = '/Users/kjo/anaconda3/bin/orca'
%config InlineBackend.figure_format = 'svg'

# For all models, the number of updates between it's time on the market and it's last security update

In [2]:
# Calculate the number of SHOULD received updates
# Calculate the number of actual received updates
# For each Carrier, Manufacture, Model

In [3]:
# Import carrier dataset and cleanup
carrierDF=pd.read_csv('../data/allCarrierData_final.csv')
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("Samgung","Samsung")
carrierDF.difference=carrierDF.difference.astype('float')
# carrierDF=carrierDF[carrierDF.Carrier!="Verizon"] # remove verizon

print(carrierDF.columns)
carrierDF.drop(columns=['Android_Level_Guessed','build','checker'],inplace=True)
# carrierDF.drop(columns=['Android_Level_Guessed','build','checker','Bulletin_Publish_Date'],inplace=True)

def upMonth(x):
    x=int(x)
    if x==12: return "01"
    elif x>=9: return str(x+1)
    else: return "0"+str(x+1)
    
# Normalized dataset    
# carrierDF['Month_Phone_Release']=carrierDF['Month_Phone_Release'].apply(lambda x: upMonth(x))
carrierDF['Phone_Published']=carrierDF.Year_Phone_Release.astype(str)+ "-"+carrierDF.Month_Phone_Release.astype(str) + "-01"
carrierDF.drop(columns=['Year_Phone_Release','Month_Phone_Release'],inplace=True)
print(carrierDF.head())

Index(['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin_Level',
       'difference', 'Android_Level_Guessed', 'build', 'Year_Phone_Release',
       'Month_Phone_Release', 'checker'],
      dtype='object')
  Carrier Manufacture               Model         Release_Date  \
0    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-03 00:00:00   
1    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-03 00:00:00   
2    AT&T     Alcatel  Alcatel IdealXCITE  2018-10-24 00:00:00   
3    AT&T     Alcatel  Alcatel IdealXCITE  2018-12-18 00:00:00   
4    AT&T  BlackBerry   BlackBerry KEYone  2017-09-26 00:00:00   

        Bulletin_Level  difference Phone_Published  
0  2018-02-01 00:00:00        61.0       2017-6-01  
1  2018-04-01 00:00:00         2.0       2017-6-01  
2  2018-08-01 00:00:00        84.0       2017-6-01  
3  2018-10-01 00:00:00        78.0       2017-6-01  
4  2017-09-01 00:00:00        25.0       2017-4-01  


In [4]:
# For each Carrier, Manufacture, Model
# 1) Calculate the number of actual received updates
# 2) Calculate the number of SHOULD received updates

carrierDF.Release_Date=pd.to_datetime(carrierDF.Release_Date)
carrierDF.Phone_Published=pd.to_datetime(carrierDF.Phone_Published)

# Me trying to troubleshoot lol
hmm=carrierDF[carrierDF.Phone_Published<pd.to_datetime("2015/08/01")]
print(hmm)
print(len(hmm.Model.unique()))
print(len(hmm))

print(len(carrierDF.Model.unique())-len(hmm.Model.unique()))
print(len(carrierDF)-len(hmm))


      Carrier Manufacture             Model Release_Date       Bulletin_Level  \
20       AT&T         HTC        Desire 626   2015-09-10  2015-09-01 00:00:00   
21       AT&T         HTC            One M9   2015-10-19  2015-10-01 00:00:00   
22       AT&T         HTC            One M9   2016-09-12  2016-08-01 00:00:00   
23       AT&T         HTC            One M9   2016-10-06  2016-09-01 00:00:00   
24       AT&T         HTC            One M9   2016-10-28  2016-10-01 00:00:00   
...       ...         ...               ...          ...                  ...   
1886  Verizon     Samsung     Galaxy Tab S2   2019-09-26  2019-08-01 00:00:01   
1903  Verizon        Sony  Xperia Z2 Tablet   2016-02-24  2016-02-01 00:00:01   
1904  Verizon        Sony        Xperia Z3v   2016-03-14  2016-03-07 00:00:01   
1913  Verizon     Verizon        Ellipsis 8   2016-05-18  2016-05-01 00:00:01   
1920  Verizon     Verizon     Ellipsis Kids   2016-06-07  2016-06-01 00:00:01   

      difference Phone_Publ

In [5]:
print(carrierDF.Carrier.unique())

['AT&T' 'Sprint' 'T-Mobile' 'Verizon']


In [6]:
 
# Normalizing/calculating frequency of updates for each phone model
carrierDF=carrierDF[carrierDF.Phone_Published>=pd.to_datetime("2015/08/01")] # Remove models released before the first bulletin
carrierDF=carrierDF[carrierDF.Phone_Published<=carrierDF.Release_Date] # Remove updates happening prior to phone release

counts=[]
for car in carrierDF.Carrier.unique():
    tmp=carrierDF[carrierDF.Carrier==car]
    for man in tmp.Manufacture.unique():
        tmp2=tmp[tmp.Manufacture==man]
        for mod in tmp2.Model.unique():
            tmp3=tmp2[tmp2.Model==mod]
            tmp3.sort_values(by=['Release_Date'],inplace=True,ascending=False) 
            totalReceived=len(tmp3) # total number of updates received for the model
            shouldReceive=tmp3.iloc[0,:] # grab the first row
            shouldReceive=(shouldReceive.Release_Date.year-shouldReceive.Phone_Published.year)*12.0+(shouldReceive.Release_Date.month+1)-shouldReceive.Phone_Published.month # account for the WHOLE last month
            if shouldReceive>0: # we skip any updates released prior to phone's release
                counts.append([car,man,mod,totalReceived,shouldReceive])
                if totalReceived>shouldReceive:
                    print(shouldReceive)
                    print(tmp3)
                    print()
            elif shouldReceive==0: 
                counts.append([car,man,mod,totalReceived,1]) # account for less than a month...
            else:
                print(shouldReceive)
                print(tmp3)
                print()
                
counts=pd.DataFrame(counts,columns=['Carrier','Manufacture','Model','Actual','Potential'])
counts['Actual']=counts['Actual'].astype(int)
counts['Potential']=counts['Potential'].astype(int)
counts['Received_Rate']=counts['Actual']/counts['Potential']
print(counts.head())

1.0
    Carrier Manufacture             Model Release_Date       Bulletin_Level  \
118    AT&T         RED  RED Hydrogen One   2018-11-01  2018-09-01 00:00:00   
119    AT&T         RED  RED Hydrogen One   2018-11-01  2018-11-01 00:00:00   

     difference Phone_Published  
118        61.0      2018-11-01  
119         0.0      2018-11-01  

  Carrier Manufacture               Model  Actual  Potential  Received_Rate
0    AT&T     Alcatel  Alcatel IdealXCITE       4         19       0.210526
1    AT&T  BlackBerry   BlackBerry KEYone      16         25       0.640000
2    AT&T      Huawei    Huawei Ascend XT       5         19       0.263158
3    AT&T     Kyocera     DuraForce PRO 2       1          6       0.166667
4    AT&T     Kyocera       DuraForce Pro       6         41       0.146341


/Users/kjo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/kjo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
print(counts.Received_Rate.mean())
print(counts.Received_Rate.median())

0.36178953239704426
0.2962962962962963


In [8]:
data=[]
# for each severity, calculate the counts on each bulletin


data.append(go.Box(
    y=counts.Received_Rate.values,
    name="Rate",
    boxpoints = 'all',
    showlegend=False,
    marker=dict(color="rgb(31, 119, 180)")
))

# Edit the layout
layout = dict(#title = "Number of Updates after Release",
              yaxis = dict(title = 'Ratio of Received Updates vs Potential',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              #xaxis = dict(title = '6 Month Period '),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='frequency-since-release-years.png')

In [20]:
# Show how the ratio alters per carrier?
for car in counts.Carrier.unique():
    tmp=counts[counts.Carrier==car]
    print(str(car)+" "+str(tmp.Received_Rate.median()))
    
print("---------------------------")
print("---------------------------")
# Show how the ratio alters per manufacturer?
for man in counts.Manufacture.unique():
    tmp=counts[counts.Manufacture==man]
    uniqueMods=len(tmp.Model.unique())
    print(str(man)+" "+str(tmp.Received_Rate.median()))
    
print("---------------------------")
print("---------------------------")
# Show how the duration alters per manufacturer?
for man in counts.Manufacture.unique():
    tmp=counts[counts.Manufacture==man]
    uniqueMods=len(tmp.Model.unique())
    print(str(man)+" "+str(tmp.Potential.max()))


AT&T 0.4444444444444444
Verizon 0.25
T-Mobile 0.3875
Sprint 0.25
---------------------------
---------------------------
Samsung 0.36363636363636365
Motorola 0.17391304347826086
Kyocera 0.14459930313588848
ASUS 0.24444444444444444
HTC 0.16666666666666666
Sonim 0.07142857142857142
LG 0.3333333333333333
ZTE 0.1111111111111111
BlackBerry 0.12
Verizon 0.21531100478468898
Google 0.39814814814814814
Coolpad 0.2625
Alcatel 0.18859649122807015
Revvl(T-Mobile) 0.3214285714285714
Huawei 0.3684210526315789
Essential 0.27586206896551724
CAT 0.2857142857142857
Palm 0.2857142857142857
KYOCERA 0.3
Razer 0.3333333333333333
Orbic 0.36363636363636365
Nokia 0.5520833333333333
OnePlus 0.75
RED 1.0
---------------------------
---------------------------
Samsung 45
Motorola 29
Kyocera 41
ASUS 21
HTC 26
Sonim 18
LG 44
ZTE 40
BlackBerry 32
Verizon 38
Google 39
Coolpad 8
Alcatel 19
Revvl(T-Mobile) 21
Huawei 19
Essential 29
CAT 7
Palm 14
KYOCERA 10
Razer 3
Orbic 11
Nokia 16
OnePlus 8
RED 3


In [10]:
# For each update, how does ratio change?
print(carrierDF.head())


  Carrier Manufacture               Model Release_Date       Bulletin_Level  \
0    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03  2018-02-01 00:00:00   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03  2018-04-01 00:00:00   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24  2018-08-01 00:00:00   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18  2018-10-01 00:00:00   
4    AT&T  BlackBerry   BlackBerry KEYone   2017-09-26  2017-09-01 00:00:00   

   difference Phone_Published  
0        61.0      2017-06-01  
1         2.0      2017-06-01  
2        84.0      2017-06-01  
3        78.0      2017-06-01  
4        25.0      2017-04-01  


In [17]:
import researchpy
carrierDF.Release_Date=pd.to_datetime(carrierDF.Release_Date)
carrierDF.Phone_Published=pd.to_datetime(carrierDF.Phone_Published)
# carrierDF['Potential']=(carrierDF.Release_Date.year-carrierDF.Phone_Published.year)*12.0+(carrierDF.Release_Date.month+1)-carrierDF.Phone_Published.month # account for the WHOLE last month


newVal_actual=[]
newVal_potential=[]
newVal_age=[]
carrierDF.sort_values(by=['Carrier','Manufacture','Model','Release_Date'],inplace=True,ascending=True) 
for car in carrierDF.Carrier.unique():
    tmp=carrierDF[carrierDF.Carrier==car]
    for man in tmp.Manufacture.unique():
        tmp2=tmp[tmp.Manufacture==man]
        for mod in tmp2.Model.unique():
            tmp3=tmp2[tmp2.Model==mod]
            for index,val in tmp3.iterrows(): # at index 0, this is the first security update since its release
                newVal_actual.append(index+1)
                newVal_potential.append((val.Release_Date.year-val.Phone_Published.year)*12.0+(val.Release_Date.month+1)-val.Phone_Published.month)
                newVal_age.append((val.Release_Date-val.Phone_Published).days)
                
                
carrierDF['Actual']=newVal_actual  
carrierDF['Potential']=newVal_potential  
carrierDF['Ratio']=carrierDF['Actual'] / carrierDF['Potential']
carrierDF['Phone_Age']=newVal_age     
    

goods=carrierDF.copy()

dfTemp=researchpy.correlation.corr_pair(goods.loc[:,['Ratio','Phone_Age']],"spearman")
print(dfTemp)
print("-------------------------------------")

# # For all manufacturers
# pd.set_option('mode.chained_assignment', None)
# for man in goods.Manufacture.unique():
#     tmp=goods[goods.Manufacture==man]
#     tmp['Ratio']=tmp['Ratio'].astype('float')
#     tmp['Phone_Age']=tmp['Phone_Age'].astype('float')
#     dfTemp=researchpy.correlation.corr_pair(tmp.loc[:,['Ratio','Phone_Age']],"spearman")
#     dfTemp['p-value']=dfTemp['p-value'].astype(float)
#     if float(dfTemp.iloc[0,1])<=0.05:
#         print(man)
#         print(dfTemp)
#         print()
# pd.reset_option('mode.chained_assignment')
    
            
# statistics. If phone age correlated with frequency       

                   r value p-value     N
Ratio & Phone_Age  -0.6722  0.0000  1461
-------------------------------------


In [13]:
# Graph noramlized frequency for each phone model... upon shared models (will be duplicated...),
    # Each device = carrier+model

# CDF for frequency... 
    # x-axis = normalized frequency
    # y-axis = percentage of counts

# for each row, count percentage of devices up to last row
counts.sort_values(by=["Received_Rate"],inplace=True)
counts=counts[counts.Received_Rate<=1.0]

total=len(counts)

runningCount=0
yaxis=[x/total for x in range(len(counts))] # fraction of devices
xaxis=[x["Received_Rate"] for index,x in counts.iterrows()]
# for index,row in counts.iterrows(): 
#     runningCount=runningCount+row["Received_Rate"]
#     xaxis.append(runningCount)

print(total)
print(yaxis[:5])
    

data=[]

data.append(go.Scatter(
    x=xaxis,
    y=yaxis,
    name="Rate",
    showlegend=False,
    marker=dict(color="rgb(31, 119, 180)"),
    mode='lines+markers'
))

# Edit the layout
layout = dict(#title = "Number of Updates after Release",
              yaxis = dict(title = 'Cumulatived Fraction of Carrier-Supported Devices',showgrid=True,zeroline=True,linecolor='rgb(219, 219, 219)',gridcolor='rgb(219, 219, 219)', mirror=True,range=[0,1],title_font=dict(size=17),tickfont=dict(size=17)),
              xaxis = dict(title = 'Normalized Update Frequency',showgrid=True,zeroline=True,linecolor='rgb(219, 219, 219)', gridcolor='rgb(219, 219, 219)', mirror=True,range=[0,1],title_font=dict(size=17),tickfont=dict(size=17)),
              plot_bgcolor='rgba(0,0,0,0)',
              font=dict(size=20)
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='frequency-all-devices.png')
pio.write_image(fig, 'all-devices-cumulative-frequency.pdf', width=1000, height=600)

# What I did:
# 1. For all devices (carrier + model), calculated the frequency ratio
    # Meaning each device could be a duplicate model (i.e., Samsung Galaxy S9 on T-Mobile and Verizon)
# 2. Sorted via frequency rate and increased y-axis as more models are added (CDF graph)
# 3. Because of data messiness, I removed ONE model that reported 2 as a frequency which shouldn't be feasible...

290
[0.0, 0.0034482758620689655, 0.006896551724137931, 0.010344827586206896, 0.013793103448275862]
